# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook has been developed from the official YOLOv6 ['Train on Custom Data' tutorial](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md). It can be used to train and evaluate YOLOv6 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the official repository.
  
Please note this uses the [3.0 release](https://github.com/meituan/YOLOv6/releases/tag/0.3.0) of the v6 repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of three parts:
1. Setup
2. Training
3. Evaluation

For additional documentation, training options and inference procedures please visit the official YOLOv6 repository.

(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

<img src="https://github.com/geezacoleman/Palmer-detection/blob/main/media/figure_2.png"/>

# 1. Setup

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import necessary packages
import torch
import random

random.seed(0)

from datetime import datetime

DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
rootDir = '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/data'
%cd '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/'

Install requirements

In [ ]:
!ls
%cd yolov6v3
!pip install -r requirements.txt

## Download pretrained models
Download the pretrained models to finetune on the custom data into the 'weights' directory within YOLOv6. They should only be downloaded once.

NOTE: These are 0.3.0 models from the [3.0 release](https://github.com/meituan/YOLOv6/releases/tag/0.3.0). For the latest models please check the official repository.

In [ ]:
!wget https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6n.pt -O weights/yolov6n.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6l.pt -O weights/yolov6l.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6n6.pt -O weights/yolov6n6.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.3.0/yolov6l6.pt -O weights/yolov6l6.pt

You can use Tensorboard to track training.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# 2. Training

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov6v3'
models = ['n', 'l', 'n6', 'l6'] # N T S M L
classes = ['1', '8']

# different folds
for model in models:
  for class_num in classes:
    for fold in range(0, 5):
      yaml_path = f"{class_num}cls_fold_{fold}.yaml"
      full_name = f'{DATE}_yolov6v3{model}_B8_F{fold}_{class_num}cls'

      print('################ STARTING {} ###################'.format(full_name))
      !python tools/train.py --workers 8 --batch 8 --device 0 --img-size {IMAGE_SIZE} --epochs 30 --conf configs/yolov6{model}_finetune.py  --data data/{data_path}  --name {full_name}

# 3. Evaluation

In [ ]:
import os
train_path = "/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov6v3/runs/train"

for run_name in os.listdir(train_path):
  if 'yolo' not in run_name:
    print(f'skipping {run_name}')
 
  else:
    fold = run_name.split("_")[-3][-1]
    class_num = run_name.split("_")[-2][:-3]
    model = run_name.split("_")[1][-1]
    
    yaml_path = f"{class_num}cls_fold_{fold}.yaml"
  
    !python tools/eval.py --device 0 --img-size {IMAGE_SIZE} --task 'test' --config-file configs/yolov6{model}_finetune_eval.py --weights runs/train/{run_name}/weights/best_ckpt.pt --data data/{data_name} --name {run_name}
